In [ ]:
import pandas as pd
import seaborn as sns
import plotly.express as px
from copy import copy
import matplotlib.pyplot as plt
import numpy as np
import plotly.figure_factory as ff
import plotly.graph_objects as go

In [ ]:
!pip install jupyterthemes


In [ ]:
from jupyterthemes import jtplot # Jupyter theme
jtplot.style(theme = 'monokai', context= 'notebook', ticks= True, grid= False)

In [ ]:
# Read the stock data file
stocks_df = pd.read_csv('stocks_dataset.csv')
stocks_df
# Getting dataframe info
stocks_df.info()
stocks_df.describe()

In [ ]:
#Noralize the stock data file, basically divide all the prices by its intial price
def Normalize(df):
    x = df.copy()
    for i in x.columns[1:]:
        x[i]=x[i]/x[i][0]
    return x

normalize_data = Normalize(stocks_df)
normalize_data


In [19]:
# Function to plot interactive plot
def interactive_plot(df, title):
    fig = px.line(title = title)
    for i in df.columns[1:]:
        fig.add_scatter(x = df['Date'], y = df[i], name = i)
    fig.show()

# Plot interactive chart
interactive_plot(stocks_df, 'Prices')

In [ ]:
# Function to calculate the daily returns
def daily_return(df):
    df_daily_return = df.copy()

    for i in df.columns[1:]:  # here, i is for columns
        for j in range(1, len(df)):  # j is for every row in column i
            df_daily_return[i][j] = ((df[i][j]-df[i][j-1])/df[i][j])*100
            df_daily_return[i][0]=0

    return df_daily_return

stocks_daily_return = daily_return(stocks_df)
print('Daily Returns:')
stocks_daily_return
# x = stocks_daily_return['TSLA']
# x

In [ ]:
# plot a scatter plot between the selected stock and the S&P500 (Market)
stocks_daily_return.plot(kind='scatter', x='sp500', y='TSLA', color='w', title='sp500 vs Tesla')

In [ ]:
# Fit a polynomial between the selected stock and the S&P500 (Poly with order = 1 is a straight line)

# beta represents the slope of the line regression line (market return vs. stock return).
# Beta is a measure of the volatility or systematic risk of a security or portfolio compared to the entire market (S&P500)
# Beta is used in the CAPM and describes the relationship between systematic risk and expected return for assets

# Beta = 1.0, this indicates that its price activity is strongly correlated with the market.
# Beta < 1, indicates that the security is theoretically less volatile than the market (Ex: Utility stocks). If the stock is included, this will make the portfolio less risky compared to the same portfolio without the stock.
# Beta > 1, indicates that the security's price is more volatile than the market. For instance, Tesla stock beta is 1.26 indicating that it's 26% more volatile than the market.
# Tech stocks generally have higher betas than S&P500 but they also have excess returns
# MGM is 65% more volatile than the S&P500!


beta_TSLA, alpha_TSLA = np.polyfit(stocks_daily_return['sp500'], stocks_daily_return['TSLA'], 1)
print('Beta for {} stock is = {} and alpha is = {}'.format('TSLA', beta_TSLA, alpha_TSLA))

# Now let's plot the scatter plot and the straight line on one plot
stocks_daily_return.plot(kind = 'scatter', x = 'sp500', y = 'TSLA', color = 'w')

# Straight line equation with alpha and beta parameters
# Straight line equation is y = beta * rm + alpha
plt.plot(stocks_daily_return['sp500'], beta_TSLA * stocks_daily_return['sp500'] + alpha_TSLA, '-', color = 'r')
plt.show()

# Let's calculate the average daily rate of return for S&P500
avg = stocks_daily_return['sp500'].mean()
print('Average is :', avg)

# Annual Market Return
# 252 days market opened in a year
rm = stocks_daily_return['sp500'].mean()*252
print('Annual Market Return is :', rm)

In [ ]:
# Assume risk free rate is zero
# Also you can use the yield of a 10-years U.S. Government bond as a risk free rate
rf=0
# Calculate return for any security (TSLA) using CAPM
ER_TLSA = rf + (beta_TSLA * (rm-rf))
ER_TLSA

# Apply CAPM formula to calculate the return for Boeing

# beta_BA, alpha_BA = np.polyfit(stocks_daily_return['sp500'], stocks_daily_return['BA'], 1)
# rf = 0
# rm = stocks_daily_return['sp500'].mean()*252

# ER_BA = rf + beta_BA*(rm - rf)
# ER_BA

In [ ]:
# Let's create a placeholder for all betas and alphas (empty dictionaries)
beta = {}
alpha = {}

# Loop on every stock daily return
for i in stocks_daily_return.columns[1:8]:

    # plot a scatter plot between each individual stock and the S&P500 (Market)
    stocks_daily_return.plot(kind = 'scatter', x = 'sp500', y = i, color = 'w')

    # Fit a polynomial between each stock and the S&P500 (Poly with order = 1 is a straight line)
    b, a = np.polyfit(stocks_daily_return['sp500'], stocks_daily_return[i], 1)

    plt.plot(stocks_daily_return['sp500'], b * stocks_daily_return['sp500'] + a, '-', color = 'r')

    beta[i] = b

    alpha[i] = a

    plt.show()


# END